In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvidia-smi

Mon Nov 28 09:42:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
%cd /content/drive/MyDrive/fakenews

/content/drive/MyDrive/fakenews


In [4]:
!pip install transformers==4.4.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 7.0 MB/s 
     |████████████████████████████████| 880 kB 61.2 MB/s 
     |████████████████████████████████| 3.3 MB 54.3 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=ea0d768718d1236c744a31c442e2a6bddb1ff0e772eebc44df95aa8950134b29
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [5]:
import pandas as pd
import numpy as np
import json,time
import os
import pdb
from tqdm import tqdm
from sklearn.metrics import accuracy_score,classification_report,roc_auc_score
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset,DataLoader,RandomSampler,SequentialSampler

from transformers import BertModel,BertConfig,BertTokenizer,AdamW,get_cosine_schedule_with_warmup

bert_path = "bert_model"
tokenizer = BertTokenizer.from_pretrained(bert_path)

In [6]:
data_split = 'train'
path = os.path.join('./data/',data_split)
file_name = os.path.join(path,'news.csv')
train_frame = pd.read_csv(file_name)
train_frame['Report Content'] = train_frame['Report Content'].apply(lambda x:x.split('##'))
data_split = 'test'
path = os.path.join('./data/',data_split)
file_name = os.path.join(path,'news.csv')
test_frame = pd.read_csv(file_name)
test_frame['Report Content'] = test_frame['Report Content'].apply(lambda x:x.split('##'))

In [7]:
input_ids,input_masks,input_types = [],[],[]
input_ids_test,input_masks_test,input_types_test = [],[],[]
labels = []
labels_test = []
maxlen = 70

for i,line in train_frame.iterrows():
    title,y = str(line['Ofiicial Account Name'])+':'+line['Title']+str(line['Report Content']),line['label']

    encode_dict = tokenizer.encode_plus(text=title,max_length=maxlen,padding='max_length',truncation=True)

    input_ids.append(encode_dict['input_ids'])
    input_types.append(encode_dict['token_type_ids'])
    input_masks.append(encode_dict['attention_mask'])

    labels.append(int(y))

for i,line in test_frame.iterrows():
    title,y = str(line['Ofiicial Account Name'])+':'+line['Title']+str(line['Report Content']),line['label']

    encode_dict = tokenizer.encode_plus(text=title,max_length=maxlen,padding='max_length',truncation=True)

    input_ids_test.append(encode_dict['input_ids'])
    input_types_test.append(encode_dict['token_type_ids'])
    input_masks_test.append(encode_dict['attention_mask'])

    labels_test.append(int(y))



input_ids, input_types, input_masks = np.array(input_ids), np.array(input_types), np.array(input_masks)
labels = np.array(labels)
print(input_ids.shape, input_types.shape, input_masks.shape, labels.shape)

input_ids_test, input_types_test, input_masks_test = np.array(input_ids_test), np.array(input_types_test), np.array(input_masks_test)
labels_test = np.array(labels_test)
print(input_ids_test.shape, input_types_test.shape, input_masks_test.shape,labels_test.shape)

(10587, 70) (10587, 70) (10587, 70) (10587,)
(10141, 70) (10141, 70) (10141, 70) (10141,)


In [8]:
# 随机打乱索引
idxes = np.arange(input_ids.shape[0])
np.random.seed(2022)   # 固定种子
np.random.shuffle(idxes)
print(idxes.shape, idxes[:10])

input_ids_train, input_ids_valid, input_ids_test = input_ids[idxes[:9000]], input_ids[idxes[9000:10587]], input_ids_test
input_masks_train, input_masks_valid, input_masks_test = input_masks[idxes[:9000]], input_masks[idxes[9000:10587]], input_masks_test
input_types_train, input_types_valid, input_types_test = input_types[idxes[:9000]], input_types[idxes[9000:10587]], input_types_test

y_train, y_valid, y_test = labels[idxes[:9000]], labels[idxes[9000:10587]], labels_test

print(input_ids_train.shape, y_train.shape, input_ids_valid.shape, y_valid.shape,

      input_ids_test.shape, y_test.shape)

(10587,) [7005 5389  563 1162 4571 4907  851 3336 7705 8574]
(9000, 70) (9000,) (1587, 70) (1587,) (10141, 70) (10141,)


In [9]:
BATCH_SIZE = 64
  # 如果会出现OOM问题，减小它
# 训练集
train_data = TensorDataset(torch.LongTensor(input_ids_train),
                           torch.LongTensor(input_masks_train),
                           torch.LongTensor(input_types_train),
                           torch.LongTensor(y_train))
train_sampler = RandomSampler(train_data)
train_loader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

# 验证集
valid_data = TensorDataset(torch.LongTensor(input_ids_valid),
                          torch.LongTensor(input_masks_valid),
                          torch.LongTensor(input_types_valid),
                          torch.LongTensor(y_valid))
valid_sampler = SequentialSampler(valid_data)
valid_loader = DataLoader(valid_data, sampler=valid_sampler, batch_size=BATCH_SIZE)

# 测试集（是没有标签的）
test_data = TensorDataset(torch.LongTensor(input_ids_test),
                          torch.LongTensor(input_masks_test),
                          torch.LongTensor(input_types_test))
test_sampler = SequentialSampler(test_data)
test_loader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

In [10]:
# 定义model
class Bert_Model(nn.Module):
    def __init__(self, bert_path, classes=2):
        super(Bert_Model, self).__init__()
        self.config = BertConfig.from_pretrained(bert_path)  # 导入模型超参数
        self.bert = BertModel.from_pretrained(bert_path)     # 加载预训练模型权重
        self.fc = nn.Linear(self.config.hidden_size, classes)  # 直接分类


    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        outputs = self.bert(input_ids, attention_mask, token_type_ids)
        out_pool = outputs[1]   # 池化后的输出 [bs, config.hidden_size]
        logit = self.fc(out_pool)   #  [bs, classes]
        return logit

In [11]:
def get_parameter_number(model):
    #  打印模型参数量
    total_num = sum(p.numel() for p in model.parameters())
    trainable_num = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return 'Total parameters: {}, Trainable parameters: {}'.format(total_num, trainable_num)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EPOCHS = 5
model = Bert_Model(bert_path).to(DEVICE)
print(get_parameter_number(model))

Total parameters: 325524482, Trainable parameters: 325524482


In [12]:
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=1e-4) #AdamW优化器
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=len(train_loader),
                                            num_training_steps=EPOCHS*len(train_loader))
# 学习率先线性warmup一个epoch，然后cosine式下降。
#加warmup（学习率从0慢慢升上去），如果把warmup去掉，可能收敛不了。

# class FocalLoss(nn.Module):
#     """Multi-class Focal loss implementation"""
#     def __init__(self, gamma=2, weight=None, reduction='mean', ignore_index=-100):
#         super(FocalLoss, self).__init__()
#         self.gamma = gamma
#         self.weight = weight
#         self.ignore_index = ignore_index
#         self.reduction = reduction

    # def forward(self, input, target):
    #     """
    #     input: [N, C]
    #     target: [N, ]
    #     """
    #     log_pt = torch.log_softmax(input, dim=1)
    #     pt = torch.exp(log_pt)
    #     log_pt = (1 - pt) ** self.gamma * log_pt
    #     loss = torch.nn.functional.nll_loss(log_pt, target, self.weight, reduction=self.reduction, ignore_index=self.ignore_index)
    #     return loss

In [13]:
# 评估模型性能，在验证集上
def evaluate(model, data_loader, device):
    model.eval()
    val_true, val_pred = [], []
    with torch.no_grad():
        for idx, (ids, att, tpe, y) in (enumerate(data_loader)):
            y_pred = model(ids.to(device), att.to(device), tpe.to(device))
            y_pred = torch.argmax(y_pred, dim=1).detach().cpu().numpy().tolist()
            val_pred.extend(y_pred)
            val_true.extend(y.squeeze().cpu().numpy().tolist())

    return accuracy_score(val_true, val_pred)  #返回accuracy

# 测试集没有标签，需要预测提交
def predict(model, data_loader, device):
    model.eval()
    val_pred = []
    with torch.no_grad():
        for idx, (ids, att, tpe) in tqdm(enumerate(data_loader)):
            y_pred = model(ids.to(device), att.to(device), tpe.to(device))
            y_pred = torch.argmax(y_pred, dim=1).detach().cpu().numpy().tolist()
            val_pred.extend(y_pred)
    return val_pred

def predict(model, data_loader, device):
    model.eval()
    val_pred = []
    with torch.no_grad():
        for idx, (ids, att, tpe) in tqdm(enumerate(data_loader)):
            y_pred = model(ids.to(device), att.to(device), tpe.to(device))
            y_pred = torch.argmax(y_pred, dim=1).detach().cpu().numpy().tolist()
            val_pred.extend(y_pred)
    return val_pred

def train_and_eval(model, train_loader, valid_loader,
                   optimizer, scheduler, device, epoch):
    best_acc = 0.0
    patience = 0
    criterion = nn.CrossEntropyLoss()
    for i in range(epoch):
        """训练模型"""
        start = time.time()
        model.train()
        print("***** Running training epoch {} *****".format(i+1))
        train_loss_sum = 0.0
        for idx, (ids, att, tpe, y) in enumerate(train_loader):
            ids, att, tpe, y = ids.to(device), att.to(device), tpe.to(device), y.to(device)
            y_pred = model(ids, att, tpe)
            loss = criterion(y_pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()   # 学习率变化

            train_loss_sum += loss.item()
            if (idx + 1) % (len(train_loader)//5) == 0:    # 只打印五次结果
                print("Epoch {:04d} | Step {:04d}/{:04d} | Loss {:.4f} | Time {:.4f}".format(
                          i+1, idx+1, len(train_loader), train_loss_sum/(idx+1), time.time() - start))
                # print("Learning rate = {}".format(optimizer.state_dict()['param_groups'][0]['lr']))

        """验证模型"""
        model.eval()
        acc = evaluate(model, valid_loader, device)  # 验证模型的性能
        ## 保存最优模型
        if acc > best_acc:
            best_acc = acc
            torch.save(model.state_dict(), "best_bert_model.pth")

        print("current acc is {:.4f}, best acc is {:.4f}".format(acc, best_acc))
        print("time costed = {}s \n".format(round(time.time() - start, 5)))

In [14]:
# 训练和验证评估
train_and_eval(model, train_loader, valid_loader, optimizer, scheduler, DEVICE, EPOCHS)

***** Running training epoch 1 *****
Epoch 0001 | Step 0028/0141 | Loss 0.6191 | Time 65.6373
Epoch 0001 | Step 0056/0141 | Loss 0.5067 | Time 133.1912
Epoch 0001 | Step 0084/0141 | Loss 0.4104 | Time 200.4344
Epoch 0001 | Step 0112/0141 | Loss 0.3591 | Time 267.8802
Epoch 0001 | Step 0140/0141 | Loss 0.3202 | Time 335.4461
current acc is 0.9231, best acc is 0.9231
time costed = 385.74772s 

***** Running training epoch 2 *****
Epoch 0002 | Step 0028/0141 | Loss 0.1426 | Time 67.8097
Epoch 0002 | Step 0056/0141 | Loss 0.1280 | Time 135.1589
Epoch 0002 | Step 0084/0141 | Loss 0.1143 | Time 202.7003
Epoch 0002 | Step 0112/0141 | Loss 0.1154 | Time 270.2465
Epoch 0002 | Step 0140/0141 | Loss 0.1112 | Time 337.8782
current acc is 0.9572, best acc is 0.9572
time costed = 365.27472s 

***** Running training epoch 3 *****
Epoch 0003 | Step 0028/0141 | Loss 0.0371 | Time 67.6677
Epoch 0003 | Step 0056/0141 | Loss 0.0382 | Time 135.2102
Epoch 0003 | Step 0084/0141 | Loss 0.0405 | Time 202.7543


In [15]:
model.load_state_dict(torch.load("best_bert_model.pth"))
pred_test = predict(model, test_loader, DEVICE)
print("\n Test Accuracy = {} \n".format(accuracy_score(y_test, pred_test)))
print("\n  roc_auc_score= {} \n".format(roc_auc_score(y_test, pred_test)))
print(classification_report(y_test, pred_test, digits=4))

159it [02:11,  1.21it/s]


 Test Accuracy = 0.9245636524997535 


  roc_auc_score= 0.8053807798521239 

              precision    recall  f1-score   support

           0     0.9400    0.9738    0.9566      8659
           1     0.8061    0.6370    0.7116      1482

    accuracy                         0.9246     10141
   macro avg     0.8731    0.8054    0.8341     10141
weighted avg     0.9205    0.9246    0.9208     10141

